In [2]:
import os
PATH=os.getcwd()
PATH='IMDB/aclImdb/'
TRN_PATH = 'train/all/'
VAL_PATH = 'test/all/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'
TRN

'IMDB/aclImdb/train/all/'

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle

In [4]:
bs,bptt = 32,50
from spacy.symbols import ORTH
my_tok = spacy.load('en')
def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

In [5]:
TEXT = data.Field(lower=True, tokenize='spacy')

In [6]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)

In [7]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [8]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(12836, 37393, 1, 20540195)

In [9]:
# 'itos': 'int-to-string'
TEXT.vocab.itos[:12]


['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is', 'in', 'it']

In [10]:
# 'stoi': 'string to int'
TEXT.vocab.stoi['the']

2

In [11]:
md.trn_ds[0].text[:12]


['waldemar',
 'deminski',
 ',',
 'the',
 'famous',
 'half',
 '-',
 'man',
 'half',
 '-',
 'wolf(in',
 'theory']

In [12]:
TEXT.numericalize([md.trn_ds[0].text[:12]])

Variable containing:
 13815
     0
     3
     2
   832
   333
    17
   142
   333
    17
     0
  3124
[torch.cuda.LongTensor of size 12x1 (GPU 0)]

In [13]:
em_sz = 300  # size of each embedding vector
nh = 500     # number of hidden activations per layer
nl = 3       # number of layers

In [14]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))


In [15]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [16]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=1, cycle_mult=1)


HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                                  
    0      4.806123   4.661396  



[4.661396]

In [17]:
learner.save_encoder('adam1_enc')
learner.load_encoder('adam1_enc')
#learner.load_cycle('adam3_10',2)


In [18]:
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))


In [19]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [56]:
m=learner.model
s="""Jim Carrey and Morgan Freeman along with Jennifer Aniston combine to make one"""



In [57]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [58]:
sample_model(m,s)


...of the most memorable films ever made . the film is a very good film , and it is a great film . <eos> 